In [1]:
import numpy as np
import pandas as pd
import re

from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sample_submission = pd.read_csv('sample_submission.csv')

In [3]:
test.sample(10)

,id,comment_text
62724,6874041e17b8ad04,Proposal \n I propose !K7 Records is merged in...
54004,59c1b16e28af332a,Bold text \n == Headline text == \n I HATE SKO...
28735,2fd559120e40d338,""" \n\n == Readers of this article want to know..."
24782,2948b4a2671ef0e9,== crazy tompson == \n\n poop
32529,3613e71d83b764f0,I don't understand. If you remove Dunlop becas...
59604,63179b38a223cc11,which means big fucking hammer
38922,409496e2fbf863d0,"Hi hater, how you doing? I know that you hate ..."
146524,f4e4fb26d875a6e5,Are you stupid? Which one are these replies un...
38782,405bbd4075b2afb5,":Since no-one has clarified this, I'm going to..."
21579,240c574e5d2f36f2,== How to organize the list and template == \n...


### The most common word

In [4]:
train_text = train['comment_text']
test_text = test['comment_text']
all_text = pd.concat([train_text, test_text])

In [5]:
def get_words(text):
    result = re.split(r'\s+|\.', text.lower())
    return result

In [6]:
total = list()
for line in all_text:
    total.extend(get_words(line))

In [7]:
count = Counter(total)
count.most_common(3)

[('', 1097088), ('the', 902873), ('to', 534076)]

Какое слово встречается чаще всего в объединенном train и test датасете? - **'the'** 

### Увеличение параметра C в Logistic regression увеличивает или уменьшает степень регуляризации?

Увеличение параметра C в Logistic regression увеличивает или уменьшает степень регуляризации?

**C** – обратный коэффициент регуляризации (тот самый C в sklearn-реализации LogisticRegression)

Ответ: **Уменьшает**

### Crossvalidation

In [8]:
class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [9]:
word_vectorizer = TfidfVectorizer() # TfidfVectorizer или CountVectorizer

word_vectorizer.fit(all_text)
train_word_features = word_vectorizer.transform(train_text)
test_word_features = word_vectorizer.transform(test_text)

In [10]:
C = 3.2

classifier = LogisticRegression(C=C,random_state=7) 

scores= []

for class_name in class_names:
    
    train_target = train[class_name]

    cv_score = np.mean(cross_val_score(classifier, train_word_features, train_target, scoring='roc_auc'))

    print('CV score for class {} is {}'.format(class_name, cv_score))
    scores.append(cv_score)
    
print('Total score is {}'.format(np.mean(scores)))



CV score for class toxic is 0.97193119880727
CV score for class severe_toxic is 0.9830390499432613
CV score for class obscene is 0.9840424143183912
CV score for class threat is 0.9859612844077553
CV score for class insult is 0.9760496985195051
CV score for class identity_hate is 0.9734729045817354
Total score is 0.979082758429653



```
C = 3.2
word_vectorizer = TfidfVectorizer()
Total score is 0.979082758429653
```

### Create submission for Kaggle

In [11]:
submission = pd.DataFrame.from_dict({'id': test['id']})

In [12]:
for class_name in class_names:  
    
    train_target = train[class_name]
    
    classifier = LogisticRegression(C=C,random_state=7)
      
    classifier.fit(train_word_features, train_target)
    
    submission[class_name] = classifier.predict_proba(test_word_features)[:, 1]

In [13]:
submission.to_csv('submission_new.csv', index=False)

In [14]:
submission.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.999616,0.177845,0.998429,0.072436,0.963661,0.311573
1,0000247867823ef7,0.002659,0.000993,0.001428,0.000165,0.003560,0.001985
2,00013b17ad220c46,0.030439,0.004444,0.013365,0.001220,0.016244,0.004082
3,00017563c3f7919a,0.001388,0.001531,0.001523,0.000510,0.002386,0.000355
4,00017695ad8997eb,0.016239,0.002618,0.005238,0.001358,0.006400,0.001839
